<h2><center><h2><center>Mini-Lab: Logistic Regression and SVMs</center></h2></center></h2>
<p><center>DS 7331</center></p>
<p><center>Created by Sadik Aman, Dawn Bowerman, Zachary Harris, Alexandre Jasserme</center></p>

<p><center>Sections of this code was adapted from: 
    <li>https://nbviewer.org/github/jakemdrew/DataMiningNotebooks/blob/master/04.%20Logits%20and%20SVM.ipynb</li>


In [1]:
from __future__ import print_function

#Libraries

import plotly
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sqlite3
from matplotlib.pyplot import figure
from sklearn.model_selection import TimeSeriesSplit
# run logistic regression and vary some parameters
from sklearn.linear_model import LogisticRegression
from sklearn import metrics as mt
from sklearn.model_selection import ShuffleSplit


df = pd.read_csv('data/owid-covid-data_modified.csv') # read in the csv file

In [2]:
# Ideas from https://scikit-learn.org/stable/modules/cross_validation.html#timeseries-cv

# Sorting data frame by date column
df['date'] = pd.to_datetime(df['date']) # Converting data columnn to datetime

df = df.sort_values(by='date', ascending=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34965 entries, 0 to 34964
Data columns (total 26 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   continent                       34965 non-null  object        
 1   location                        34965 non-null  object        
 2   date                            34965 non-null  datetime64[ns]
 3   new_cases                       34965 non-null  float64       
 4   new_cases_smoothed              34965 non-null  float64       
 5   new_deaths                      34965 non-null  float64       
 6   new_deaths_smoothed             34965 non-null  float64       
 7   reproduction_rate               34965 non-null  float64       
 8   new_vaccinations_smoothed       34965 non-null  float64       
 9   new_people_vaccinated_smoothed  34965 non-null  float64       
 10  stringency_index                34965 non-null  float64       
 11  po

Using the table above, we eliminated particular variables because they would not be appropriate for the model. The dropped variables are as follows:

*Unnamed: 0*: This variable was an index in the original dataset. This has no prediction value for the model.

*new_deaths*: This variable was eliminated due to high correlation with new_deaths_range, our response variable.

*new_deaths_smoothed*: This variable was eliminated for the same reason new_deaths was eliminated. 

*new_cases*: This variable was eliminated due to high correlation with new_deaths. 

*continent*: The variable type for this attribute is object.

*location*: The variable type for this attribute is object.

*stringency_range*:The variable type for this attribute is object.

*new_cases_range*: This variable was created as a potential reponse variable. The variable type for this attribute is object.  

*date*: While this variable was used to sort the dataset initially, it has to be eliminated when incorporating the dataset into the model due to the variable type.

*new_people_vaccinated_smoothed*: This variable was eliminated due to high correlation with new_vaccinations_smoothed.

*population_density*: This variable was eliminated due to high correlation with population.

*aged_70_older*: This variable was eliminated due to high correlation with aged_65_older. 

*cardiovasc_death_rate*: This variable had a low logistic regression weight.







In [3]:


float_df = df.drop(["new_deaths", 
                    "new_deaths_smoothed",
                    "new_cases",
                    "continent",
                    "location",
                    "stringency_range",
                    "new_cases_range",
                    "date",
                    "new_people_vaccinated_smoothed",
                    "population_density",
                    "aged_70_older",
                    "cardiovasc_death_rate"], axis=1)

float_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34965 entries, 0 to 34964
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   new_cases_smoothed          34965 non-null  float64
 1   reproduction_rate           34965 non-null  float64
 2   new_vaccinations_smoothed   34965 non-null  float64
 3   stringency_index            34965 non-null  float64
 4   population                  34965 non-null  int64  
 5   median_age                  34965 non-null  float64
 6   aged_65_older               34965 non-null  float64
 7   gdp_per_capita              34965 non-null  float64
 8   diabetes_prevalence         34965 non-null  float64
 9   handwashing_facilities      34965 non-null  float64
 10  hospital_beds_per_thousand  34965 non-null  float64
 11  life_expectancy             34965 non-null  float64
 12  human_development_index     34965 non-null  float64
 13  new_deaths_range            349

In [4]:
# to predict the X and y data as follows:
if 'new_deaths_range' in float_df:
    y = float_df['new_deaths_range'].values # get the labels
    del float_df['new_deaths_range'] # get rid of the class label
    X = float_df.values # use everything else to predict
       ## X and y are now numpy matrices, by calling 'values' on the pandas data frames they 
    #    have converted into simple matrices to use with scikit learn


In [5]:
# create a reusable logisitic regression object
#   setup the object with different learning parameters and constants
lr_clf = LogisticRegression(penalty='l2', C=1.0, class_weight=None, max_iter=500, solver='lbfgs', multi_class='multinomial' ) # get object


In [6]:
iter_num=0
# the indices are the rows used for training and testing in each iteration

cv_object = TimeSeriesSplit( n_splits=3)

# TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None)
for train_index, test_index in cv_object.split(X):
   
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index] 

    lr_clf.fit(X_train,y_train)  # train object
    y_hat = lr_clf.predict(X_test) # get test set precitions
    
# accuracy and confusion matrix for this iterations of training/testing
    acc = mt.accuracy_score(y_test,y_hat)
    conf = mt.confusion_matrix(y_test,y_hat,labels=["high","medium","low"] )
    print("====Iteration",iter_num," ====")
    print("accuracy", acc )
    print("confusion matrix\n",conf)
    iter_num+=1

====Iteration 0  ====
accuracy 0.5839148838805629
confusion matrix
 [[2314  173  615]
 [ 364  381 1739]
 [ 265  481 2409]]
====Iteration 1  ====
accuracy 0.6529001258437249
confusion matrix
 [[2206  299  463]
 [ 205  256 1785]
 [ 187   95 3245]]
====Iteration 2  ====
accuracy 0.6463791328223315
confusion matrix
 [[2191  106  368]
 [ 397   73 1754]
 [ 414   52 3386]]


In [7]:
#interpret the weights

# iterate over the coefficients
weights = lr_clf.coef_.T # take transpose to make a column vector
variable_names = float_df.columns
for coef, name in zip(weights,variable_names):
    print(name, 'has weight of', coef[0])


new_cases_smoothed has weight of 0.0007373063207444708
reproduction_rate has weight of -2.658740724130506e-09
new_vaccinations_smoothed has weight of 2.1578539238857453e-07
stringency_index has weight of 1.0895031438689603e-06
population has weight of -1.0040771442719033e-09
median_age has weight of 4.802091558412188e-07
aged_65_older has weight of 3.502564016756909e-07
gdp_per_capita has weight of -3.3441886256070424e-05
diabetes_prevalence has weight of -4.063690261350669e-08
handwashing_facilities has weight of 1.4052003916775789e-06
hospital_beds_per_thousand has weight of 6.069862620106982e-08
life_expectancy has weight of 2.5563753238383275e-08
human_development_index has weight of 6.113093659710626e-09


After examining the logistic regression weights, we realized that new_cases_smoothed has a heavy influence on the model. This could create bias among the predictors. So, we dropped new_cases_smoothed from the dataset to compare both models. 

In [8]:
float_df = float_df.drop(["new_cases_smoothed"], axis=1)

In [9]:
# to predict the X and y data as follows:
if 'new_deaths_range' in float_df:
    y = float_df['new_deaths_range'].values # get the labels
    del float_df['new_deaths_range'] # get rid of the class label
    X = float_df.values # use everything else to predict
       ## X and y are now numpy matrices, by calling 'values' on the pandas data frames they 
    #    have converted into simple matrices to use with scikit learn



In [19]:

iter_num=0
# the indices are the rows used for training and testing in each iteration

cv_object = TimeSeriesSplit( n_splits=3)

# TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None)
for train_index, test_index in cv_object.split(X):
   
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index] 

    lr_clf.fit(X_train,y_train)  # train object
    y_hat = lr_clf.predict(X_test) # get test set precitions
    
# accuracy and confusion matrix for this iterations of training/testing
    acc = mt.accuracy_score(y_test,y_hat)
    conf = mt.confusion_matrix(y_test,y_hat, labels=["high","medium",""])
    print("====Iteration",iter_num," ====")
    print("accuracy", acc )
    print("confusion matrix\n",conf)
    iter_num+=1

====Iteration 0  ====
accuracy 0.5839148838805629
confusion matrix
 [[2314  173    0]
 [ 364  381    0]
 [   0    0    0]]
====Iteration 1  ====
accuracy 0.6529001258437249
confusion matrix
 [[2206  299    0]
 [ 205  256    0]
 [   0    0    0]]
====Iteration 2  ====
accuracy 0.6463791328223315
confusion matrix
 [[2191  106    0]
 [ 397   73    0]
 [   0    0    0]]


In [11]:
float_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34965 entries, 0 to 34964
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   reproduction_rate           34965 non-null  float64
 1   new_vaccinations_smoothed   34965 non-null  float64
 2   stringency_index            34965 non-null  float64
 3   population                  34965 non-null  int64  
 4   median_age                  34965 non-null  float64
 5   aged_65_older               34965 non-null  float64
 6   gdp_per_capita              34965 non-null  float64
 7   diabetes_prevalence         34965 non-null  float64
 8   handwashing_facilities      34965 non-null  float64
 9   hospital_beds_per_thousand  34965 non-null  float64
 10  life_expectancy             34965 non-null  float64
 11  human_development_index     34965 non-null  float64
dtypes: float64(11), int64(1)
memory usage: 3.5 MB


After dropping the new_cases_smoothed variable, we noticed that the model had increased difficulty classifying the medium death_range. We believe this is due to the model losing its strongest predictor.

In [12]:
# Once normalized, te attributes weights should have magnitudes that reflect their predictive power in the logistic regression model.

from sklearn.preprocessing import StandardScaler

In [13]:
# scale attributes by the training set
scl_obj = StandardScaler()
scl_obj.fit(X_train) # find scalings for each column that make this zero mean and unit std
# the line of code above only looks at training data to get mean and std and we can use it 
# to transform new feature data

X_train_scaled = scl_obj.transform(X_train) # apply to training


In [14]:
X_test_scaled = scl_obj.transform(X_test) # apply those means and std to the test set (without snooping at the test set values)

# train the model just as before
lr_clf = LogisticRegression(penalty='l2', C=1.0, class_weight=None, max_iter=500, solver='lbfgs', multi_class='multinomial' )
lr_clf.fit(X_train_scaled,y_train)  # train object

y_hat = lr_clf.predict(X_test_scaled) # get test set precitions

acc = mt.accuracy_score(y_test,y_hat)
conf = mt.confusion_matrix(y_test,y_hat, labels = ["high","medium","low"])
print('accuracy:', acc )
print(conf )




accuracy: 0.6590779087060977
[[2135  146  384]
 [ 221  449 1554]
 [ 215  460 3177]]


Compared to the previous two models, scaling the X_train data seems to improve the overall model. Results from this model show improved accuracy. 

In [15]:
# sort these attributes and spit them out
zip_vars = zip(lr_clf.coef_.T,float_df.columns) # combine attributes
for coef, name in zip_vars:
    print(name, 'has weight of', coef[0]) # now print them out


reproduction_rate has weight of 6.811952983286349
new_vaccinations_smoothed has weight of -0.16465299913189227
stringency_index has weight of 0.03985355901441289
population has weight of 0.19786080264613945
median_age has weight of 0.04494986691217253
aged_65_older has weight of 0.20939868544000886
gdp_per_capita has weight of 0.14792270710585903
diabetes_prevalence has weight of -0.7476756652436289
handwashing_facilities has weight of -0.029519022480499427
hospital_beds_per_thousand has weight of 0.11663009949696555
life_expectancy has weight of -0.09175642504238875
human_development_index has weight of -0.08184057693604677


In [16]:
from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('ggplot')


weights = pd.Series(lr_clf.coef_[0],index=float_df.columns)
weights.plot(kind='bar')
plt.show()


ValueError: Length of values (13) does not match length of index (12)

In [ ]:
# Using Pipeline
from sklearn.pipeline import Pipeline
# you can apply the StandardScaler function inside of the cross-validation loop 
#  but this requires the use of PipeLines in scikit. 
#  A pipeline can apply feature pre-processing and data fitting in one compact notation
#  Here is an example!

std_scl = StandardScaler()
lr_clf = LogisticRegression(penalty='l2', C=0.02, solver='liblinear') 

# create the pipline
piped_object = Pipeline([('scale', std_scl),  # do this
                         ('logit_model', lr_clf)]) # and then do this

weights = []
# run the pipline cross validated
for iter_num, (train_indices, test_indices) in enumerate(cv_object.split(X,y)):
    piped_object.fit(X[train_indices],y[train_indices])  # train object
    # it is a little odd getting trained objects from a  pipeline:
    weights.append(piped_object.named_steps['logit_model'].coef_[0])
    

weights = np.array(weights)
import plotly
plotly.offline.init_notebook_mode() # run at the start of every notebook

error_y=dict(
            type='data',
            array=np.std(weights,axis=0),
            visible=True
        )

graph1 = {'x': float_df.columns,
          'y': np.mean(weights,axis=0),
    'error_y':error_y,
       'type': 'bar'}

fig = dict()
fig['data'] = [graph1]
fig['layout'] = {'title': 'Logistic Regression Weights, with error bars'}

plotly.offline.iplot(fig)


In [ ]:
#  For more improvement and guarding against overfitting

Xnew = float_df[['handwashing_facilities','hospital_beds_per_thousand']].values

weights = []
# run the pipline crossvalidated
for iter_num, (train_indices, test_indices) in enumerate(cv_object.split(Xnew,y)):
    piped_object.fit(Xnew[train_indices],y[train_indices])  # train object
    weights.append(piped_object.named_steps['logit_model'].coef_[0])
    
weights = np.array(weights)

error_y=dict(
            type='data',
            array=np.std(weights,axis=0),
            visible=True
        )

graph1 = {'x': ['handwashing_facilities','hospital_beds_per_thousand'],
          'y': np.mean(weights,axis=0),
    'error_y':error_y,
       'type': 'bar'}

fig = dict()
fig['data'] = [graph1]
fig['layout'] = {'title': 'Logistic Regression Weights, with error bars'}

plotly.offline.iplot(fig)



>**Support Vector Machines**

In [ ]:
# cross validation loop and set the training and testing variable for one single iteration
for train_indices, test_indices in cv_object.split(X,y): 
    
    X_train = X[train_indices]
    y_train = y[train_indices]
    
    X_test = X[test_indices]
    y_test = y[test_indices]
    
X_train_scaled = scl_obj.transform(X_train) # apply to training
X_test_scaled = scl_obj.transform(X_test) 
# SVMs on the data and play with the parameters and kernels
from sklearn.svm import SVC

# train the model
svm_clf = SVC(C=0.5, kernel='rbf', decision_function_shape='ovo', degree=3, gamma='auto') # get object
svm_clf.fit(X_train_scaled, y_train)  # train object

y_hat = svm_clf.predict(X_test_scaled) # get test set predictions

acc = mt.accuracy_score(y_test,y_hat)
conf = mt.confusion_matrix(y_test,y_hat)
print('accuracy:', acc )
print(conf)



In [ ]:
# look at the support vectors
print(svm_clf.support_vectors_.shape)
print(svm_clf.support_.shape)
print(svm_clf.n_support_ )


In [ ]:
# if using linear kernel, these make sense to look at (not otherwise, why?)
# print(svm_clf.coef_)
# weights = pd.Series(svm_clf.coef_[0],index=float_df.columns)
# weights.plot(kind='bar')


In [ ]:
# make a dataframe of the training data
df_tested_on = float_df.iloc[train_indices].copy() # saved from above, the indices chosen for training
# get the support vectors from the trained model
df_support = df_tested_on.iloc[svm_clf.support_,:].copy()

df_support['new_deaths_range'] = y[svm_clf.support_] # add back in the 'new_deaths_range' Column to the pandas dataframe
float_df['new_deaths_range'] = y # also add it back in for the original data
df_support.info()


In [ ]:
# statistics of the attributes
from pandas.plotting import boxplot

# group the original data and the support vectors
df_grouped_support = df_support.groupby(['new_deaths_range'])
df_grouped = float_df.groupby(['new_deaths_range'])

# plot KDE of Different variables
vars_to_plot = ['reproduction_rate','new_vaccinations_smoothed','stringency_index','human_development_index']

for v in vars_to_plot:
    plt.figure(figsize=(20,4))
    # plot support vector stats
    plt.subplot(1,2,1)
    ax = df_grouped_support[v].plot.kde() 
    plt.legend(['low','medium','high'])
    plt.title(v+' (Instances chosen as Support Vectors)')
    
    # plot original distributions
    plt.subplot(1,2,2)
    ax = df_grouped[v].plot.kde() 
    plt.legend(['low','medium','high'])
    plt.title(v+' (Original)')
